In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import xgboost as xgb
from xgboost import XGBClassifier

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
# da1_train = pd.read_csv('/Users/divyeshbhatt/Downloads/Datas/Kaggle_Contests/Classification/Titan/Spaceship/train.csv')
# da1_Valid = pd.read_csv('/Users/divyeshbhatt/Downloads/Datas/Kaggle_Contests/Classification/Titan/Spaceship/test.csv')

In [5]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Function to preprocess data
def preprocess_data(df):
    # Handle missing values for categorical features
    for column in ['HomePlanet', 'Destination', 'VIP']:
        mode = df[column].mode()[0]
        df[column].fillna(mode, inplace=True)

    # Handle the Cabin feature by splitting and then imputing
    df['Cabin_Deck'] = df['Cabin'].apply(lambda x: str(x).split('/')[0])
    df['Cabin_Num'] = df['Cabin'].apply(lambda x: str(x).split('/')[1] if len(str(x).split('/')) > 1 else x)
    df['Cabin_Side'] = df['Cabin'].apply(lambda x: str(x).split('/')[2] if len(str(x).split('/')) > 2 else x)
    for column in ['Cabin_Deck', 'Cabin_Num', 'Cabin_Side']:
        mode = df[column].mode()[0]
        df[column].fillna(mode, inplace=True)
    df.drop(columns=['Cabin', 'Name'], inplace=True)

    # Impute missing values for numerical features
    for column in ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']:
        if column == 'Age':
            median = df[column].median()
            df[column].fillna(median, inplace=True)
        else:
            df[column].fillna(0, inplace=True)

    # Encoding categorical variables
    label_encoder = LabelEncoder()
    for column in ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Cabin_Deck', 'Cabin_Side']:
        df[column] = label_encoder.fit_transform(df[column])

    # Convert Cabin_Num to numeric
    df['Cabin_Num'] = pd.to_numeric(df['Cabin_Num'], errors='coerce')
    df['Cabin_Num'].fillna(df['Cabin_Num'].median(), inplace=True)

    return df

# Load the training data
train_data = pd.read_csv('/Users/divyeshbhatt/Downloads/Datas/Kaggle_Contests/Classification/Titan/Spaceship/train.csv')

# Preprocess the training data
train_data_processed = preprocess_data(train_data)

# Splitting the data into features (X) and target variable (y)
X = train_data_processed.drop(['PassengerId', 'Transported'], axis=1)
y = train_data_processed['Transported']

# Splitting into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Training the XGBoost Model
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

# Predicting on the validation set and evaluating the model
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy}")

# Load and preprocess the test data
test_data = pd.read_csv('/Users/divyeshbhatt/Downloads/Datas/Kaggle_Contests/Classification/Titan/Spaceship/test.csv')
test_data_processed = preprocess_data(test_data)

# Prepare test data for prediction
X_test = test_data_processed.drop(['PassengerId'], axis=1)

# Making predictions on test data
test_predictions = model.predict(X_test)

# Preparing the submission file
submission = pd.DataFrame({
    'PassengerId': test_data_processed['PassengerId'],
    'Transported': test_predictions
})

# Save submission file
submission.to_csv('submission.csv', index=False)




Validation Accuracy: 0.8016101207590569
